In [ ]:
%matplotlib inline
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import os
from scipy.spatial.transform import Rotation as R
import random
import os.path as osp

In [ ]:
def rodrigues(r: np.ndarray) -> np.ndarray:
    if r.size == 3: return R.from_rotvec(r.squeeze()).as_matrix()
    else: return R.from_matrix(r).as_rotvec().reshape((3, 1))

In [ ]:
def getRot(transform: np.ndarray) -> np.ndarray:
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return rodrigues(transform[:3])
    elif transform.shape == (3, 4) or transform.shape == (4, 4):
        return transform[:3, :3]
    else:
        return None

In [ ]:
def getTr(transform: np.ndarray) -> np.ndarray:
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return transform[3:6].reshape((3, 1))
    elif transform.shape == (3, 4) or transform.shape == (4, 4):
        return transform[:3, :3].reshape((3, 1))
    else:
        return None

In [ ]:
def inverseTransform(transform: np.ndarray) -> np.ndarray:
    R, tr = getRot(transform), getTr(transform)
    R_inv = R.transpose()
    tr_inv = -R_inv.dot(tr)
    if transform.size == 6:
        r_inv = rodrigues(R_inv)
        return np.concatenate((r_inv, tr_inv), axis=0) # (6, 1) vector
    else:
        return np.concatenate((R_inv, tr_inv), axis=1) # (3, 4) matrix

In [ ]:
def toPoseMatrix(transform: np.ndarray) -> np.ndarray:
    if transform.size == 6:
        return np.concatenate(
            (getRot(transform), getTr(transform)), 1)
    else:
        return transform[:3, :4]

In [ ]:
def mergedTransform(t2: np.ndarray, t1: np.ndarray) -> np.ndarray: # T2 * T1
    R1, tr1 = getRot(t1), getTr(t1)
    R2, tr2 = getRot(t2), getTr(t2)
    R = np.matmul(R2, R1)
    tr = R2.dot(tr1) + tr2
    if t1.size == 6:
        rot = rodrigues(R)
        return np.concatenate((rot, tr), axis=0)
    else:
        return np.concatenate((R, tr), axis=1)

In [ ]:
def applyTransform(transform: np.ndarray, P: 'np.ndarray') \
        -> 'np.ndarray':
    R, tr = getRot(transform), getTr(transform)
    return R.dot(P) + tr

In [ ]:
def skew(tr: np.ndarray) -> np.ndarray:
    skew_tr = np.array([
                [0.     ,-tr[2,0],   tr[1,0]],
                [tr[2,0],      0.,  -tr[0,0]],
                [-tr[1,0], tr[0,0],        0.]])
    return skew_tr

In [ ]:
def loadPoseTxtFile(path: str, delimiter = ' '):
    if not osp.exists(path):
        return None
    data = np.loadtxt(path, delimiter=delimiter)
    fidxs = data[:, 0].astype(int).flatten()
    poses = data[:, 1:7].T.reshape((6, -1))
    timestamps = data[:, 7].astype(int).flatten()
    nposes = poses.shape[1]
    return (fidxs, poses, timestamps)

In [ ]:
def FundamentalMatrix(c2w1, c2w2, K):
    w2c1 = inverseTransform(c2w1)
    relpose = mergedTransform(w2c1,c2w2)
    relposePrj = inverseTransform(relpose)
    tr = getTr(relposePrj)
    rot = getRot(relposePrj)
    tr_skew = skew(tr)
    E = tr_skew@ rot
    F = np.linalg.inv(K.T)@E@np.linalg.inv(K)
    return F

In [ ]:
def drawlines(img1,img2,lines,pts1):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    _,w = img1.shape[:2]
    # for pt1 in pts1:
    for r,pt1 in zip(lines,pts1):
        color = tuple(np.random.randint(0,255,3).tolist())
        # x0,y0 = map(int, [0, -r[2]/r[1] ])
        # x1,y1 = map(int, [w, -(r[2]+r[0]*w)/r[1] ])
        # img2 = cv.line(img2, (x0,y0), (x1,y1), color,3)
        img2 = cv.polylines(img2, [pt1],False, color, thickness=1)
        img1 = cv.circle(img1,tuple(pt1),8,color,-1)
        # img2 = cv.circle(img2,tuple(pt2),8,color,-1)
    return img1,img2

In [ ]:
base_path = "/data2/chaerin/nerf_hyu_shared/box3d"
# pose_file = os.path.join(base_path, "trajectory_train.txt")
pose_file = os.path.join(base_path, "slam_poses.txt")
# pose_file = os.path.join(base_path, "all_frame_poses.txt")
# c2r = np.array([0.81310401, 0., 3.03454548, 0.13, 0., 0.1])
c2r = np.array([0., 0., 0., 0., 0., 0.])
image_path = os.path.join(base_path,"cam1")
image_dir = sorted(os.listdir(image_path))
fx = 370.94472982794093
fy = 370.94472982794093
cx = 669.02964375147337
cy = 585.87130046898005
K = np.array([
    [fx, 0., cx],
    [0., fy, cy],
    [0., 0., 1.]
])
fidx,pose, _ = loadPoseTxtFile(pose_file)
fidx = fidx.tolist()
cur_fidx = 523
next_fidx = 716
cur_idx = fidx.index(cur_fidx)
next_idx = fidx.index(next_fidx)
r2w1 = pose[:,cur_idx]
r2w2 = pose[:,next_idx]
c2w1 = mergedTransform(r2w1, c2r)
c2w2 = mergedTransform(r2w2, c2r)
F = FundamentalMatrix(c2w1,c2w2,K)
# img1 = cv.imread(osp.join(image_path, image_dir[cur_fidx]))
img1 = cv.imread(osp.join(image_path, '00523.png'))
# img2 = cv.imread(osp.join(image_path, image_dir[next_fidx]))
img2 = cv.imread(osp.join(image_path, '00716.png'))
plt.subplot(1,2,1),plt.imshow(img1)
plt.subplot(1,2,2),plt.imshow(img2)
pts = plt.ginput(3)
pts = np.int32(pts)
lines = []
pts_homo = []
for pt in pts:
    pt_homo = np.array([pt[0], pt[1], 1.]).reshape(3,1)
    pts_homo.append(pt_homo)
    lines.append(F@pt_homo)
img3,img4 = drawlines(img1,img2, lines, pts)
plt.subplot(1,2,1),plt.imshow(img3)
plt.subplot(1,2,2),plt.imshow(img4)
plt.show()